In [11]:
import pandas
import numpy
pandas.options.mode.chained_assignment = None

In [12]:
circulatory_mortality = pandas.read_csv("input/Circulatory Mortality.csv")
respiratory_mortality = pandas.read_csv("input/Respiratory Mortality.csv")
all_causes_mortality = pandas.read_csv("input/All Cause Mortality.csv")
all_data = [circulatory_mortality, respiratory_mortality, all_causes_mortality]

In [13]:
def convert_ids_to_int(df):
    df = df[df["State ANSI"].notnull()]
    df["STATEFP"] = pandas.to_numeric(df["State ANSI"], downcast="integer")
    df["COUNTYFP"] = pandas.to_numeric(df["County ANSI"], downcast="integer")
    return df.drop(columns=["State ANSI", "County ANSI"])

def calculate_mortality(df):
    df["mort_0-5"] = df["deaths_0-5"] / df["Population_0-5"].replace({ 0 : numpy.nan })
    df["mort_5-25"] = df["deaths_5-25"] / df["Population_5-25"].replace({ 0 : numpy.nan })
    df["mort_25+"] = df["deaths_25+"] / df["Population_25+"].replace({ 0 : numpy.nan })

In [14]:
circulatory_mortality = convert_ids_to_int(circulatory_mortality)
respiratory_mortality = convert_ids_to_int(respiratory_mortality)
all_causes_mortality = convert_ids_to_int(all_causes_mortality)

rename_map = {
    "Deaths_0-5": "deaths_0-5",
    "Deaths_25+": "deaths_25+",
    "Deaths_5-25": "deaths_5-25",
}

circulatory_mortality = circulatory_mortality.rename(columns=rename_map)
respiratory_mortality = respiratory_mortality.rename(columns=rename_map)
all_causes_mortality = all_causes_mortality.rename(columns=rename_map)

calculate_mortality(circulatory_mortality)
calculate_mortality(respiratory_mortality)
calculate_mortality(all_causes_mortality)

circ_and_resp = circulatory_mortality.merge(respiratory_mortality, on=["STATEFP", "COUNTYFP", "Population_0-5", "Population_25+", "Population_5-25"], suffixes=["_circ", "_resp"])
all_data = all_causes_mortality.merge(circ_and_resp, on=["STATEFP", "COUNTYFP", "Population_0-5", "Population_25+", "Population_5-25"])

In [20]:
all_data.to_csv("output/mortality.csv", index=None)
all_data

,deaths_0-5,deaths_25+,deaths_5-25,Population_0-5,Population_25+,Population_5-25,STATEFP,COUNTYFP,mort_0-5,mort_5-25,...,deaths_5-25_circ,mort_0-5_circ,mort_5-25_circ,mort_25+_circ,deaths_0-5_resp,deaths_25+_resp,deaths_5-25_resp,mort_0-5_resp,mort_5-25_resp,mort_25+_resp
0,0.0,510.0,0.0,3191.0,37332.0,14893.0,1,1,0.000000,0.000000,...,0.0,0.0,0.0,0.004206,0.0,46.0,0.0,0.0,0.0,0.001232
1,11.0,1933.0,23.0,11660.0,147731.0,49172.0,1,3,0.000943,0.000468,...,0.0,0.0,0.0,0.004271,0.0,173.0,0.0,0.0,0.0,0.001171
2,0.0,235.0,0.0,1352.0,18326.0,6287.0,1,5,0.000000,0.000000,...,0.0,0.0,0.0,0.004420,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,224.0,0.0,1274.0,16121.0,5248.0,1,7,0.000000,0.000000,...,0.0,0.0,0.0,0.004528,0.0,13.0,0.0,0.0,0.0,0.000806
4,0.0,675.0,0.0,3353.0,39867.0,14484.0,1,9,0.000000,0.000000,...,0.0,0.0,0.0,0.004615,0.0,103.0,0.0,0.0,0.0,0.002584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,0.0,298.0,0.0,3107.0,28510.0,12548.0,56,37,0.000000,0.000000,...,0.0,0.0,0.0,0.002911,0.0,12.0,0.0,0.0,0.0,0.000421
3136,0.0,74.0,0.0,1209.0,17488.0,4494.0,56,39,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3137,0.0,137.0,0.0,1634.0,13119.0,6020.0,56,41,0.000000,0.000000,...,0.0,0.0,0.0,0.001906,0.0,0.0,0.0,0.0,0.0,0.000000
3138,0.0,78.0,0.0,474.0,5722.0,2039.0,56,43,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
